In [1]:
# add autoreload
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import gc

from typing import List, Tuple, Dict, Union

In [2]:
import torch

In [3]:
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction

from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import benedict
import random

In [4]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
    print(f"Memory Allocated: {torch.cuda.memory_allocated()}")
    print(f"Max memory Allocated: {torch.cuda.max_memory_allocated()}") 
    print(f"Memory reserved: {torch.cuda.memory_reserved()}")
    print(f"Max memory reserved: {torch.cuda.max_memory_reserved()}")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA RTX 4000 Ada Generation
Memory Allocated: 0
Max memory Allocated: 0
Memory reserved: 0
Max memory reserved: 0


In [6]:
deabbreviate = False
filter_reports = True
Class = 'rv_syst_func' # lv_dil, rv_dil, pe, aortic_regurgitation, diastolic_dysfunction, lv_syst_func, aortic_stenosis, rv_syst_func, mitral_regurgitation,tricuspid_regurgitation,wma
FLAG_TERMS = ['uitslag zie medische status', 'zie status', 'zie verslag status', 'slecht echovenster', 'echo overwegen', 'ge echo',
              'geen echovenster', 'geen beoordeelbaar echo', 'geen beoordeelbare echo', 'verslag op ic']
SAVE_TERMS = ['goed', 'geen', 'normaal', 'normale']
use_multilabel = False
reduce_labels = True
num_epochs = 5

MULTILABELS = {'Mild': ['Mild', 'Present'], 
               'Severe': ['Severe', 'Present'],
               'Moderate': ['Moderate', 'Present'],
               'Normal': ['Normal'],
               'No label': ['No label'],
               'Present': ['Present'],
               }

Target_remapper = {'No label': 'No label', 
                   'Normal': 'Normal',
                   'Mild': 'Not normal',
                   'Moderate': 'Not normal',
                   'Severe': 'Not normal',
                   'Present':  'Not normal'
                   }


#Target_remapper = {'No label': 'Normal', 
#                   'Normal': 'Normal',
#                   'Mild': 'Not normal',
#                   'Moderate': 'Not normal',
#                   'Severe': 'Not normal',
#                   'Present':  'Not normal'
#                   }

In [7]:
#Add the src folder to the path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import deabber, echo_utils

In [8]:
if deabbreviate:
    ABBREVIATIONS = benedict.benedict("../assets/abbreviations.yml")

In [9]:
plt.style.use('ggplot')
def plot_history(history, val=0):
    acc = history.history['accuracy']
    if val == 1:
        val_acc = history.history['val_accuracy'] # we can add a validation set in our fit function with nn
    loss = history.history['loss']
    if val == 1:
        val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    if val == 1:
        plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.title('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    if val == 1:
        plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title('Loss')
    plt.legend()

In [10]:
def compute_metrics_binomial(logits_and_labels, averaging='macro'):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average = averaging)
  prec = precision_score(labels, predictions, average = averaging)
  rec = recall_score(labels, predictions, average = averaging)
  return {
          'accuracy': acc, 
          'f1_score': f1,
          'precision': prec,
          'recal': rec
          }

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    
    
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    prec_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    prec_weighted = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
    recall_macro = recall_score(y_true=y_true, y_pred=y_pred, average='macro')
    recall_weighted = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')    
    
    try:
        roc_auc_weighted = roc_auc_score(y_true, probs, average = 'weighted')
        roc_auc_macro = roc_auc_score(y_true, probs, average = 'macro')
    except:
        roc_auc_weighted = None
        roc_auc_macro = None
    
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1_macro': f1_macro,
               'f1_weighted': f1_weighted,
               'prec_macro': prec_macro,
               'prec_weighted': prec_weighted,
               'recall_macro': recall_macro,
               'recall_weighted': recall_weighted,
               'roc_auc_macro': roc_auc_macro,
               'roc_auc_weighted': roc_auc_weighted,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result
     


In [11]:
def one_hot_encode(labels, num_classes):
    # labels should be a list or a 1D tensor
    one_hot = torch.zeros((len(labels), num_classes))
    rows = torch.arange(len(labels))
    one_hot[rows, labels] = 1
    return one_hot

def _mlabel_tuple_creator(x: List[int],
                          multilabels:Dict[int,List[int]],
                          num_classes: int=None)\
                          ->List[Tuple[int,...]]:
                              
    res = [(_sc for _sc in multilabels[sc]) for sc in x]
    return res

def multi_hot_encoding(x: List[int], 
                       multilabels: Union[Dict[int,List[int]], None]=None,
                       num_classes: int=None)\
                           ->torch.Tensor:    
    if multilabels is None:
        return one_hot_encode(x, num_classes=num_classes)
    else:
        return torch.Tensor(MultiLabelBinarizer(classes=range(num_classes))\
                    .fit_transform(_mlabel_tuple_creator(x,multilabels)))

# Load documents

In [12]:
tokenizer = AutoTokenizer.from_pretrained("CLTL/MedRoBERTa.nl")

c:\Users\bes3\AppData\Local\pypoetry\Cache\virtualenvs\echolabeler-DyKQcQCO-py3.10\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
os.chdir('T://lab_research/RES-Folder-UPOD/Echo_label/E_ResearchData/2_ResearchData')

In [14]:
labeled_documents = pd.read_json(f"./echo_doc_labels/{Class}.jsonl", lines=True)

In [15]:
labeled_documents.label.value_counts()

label
No label    2538
Normal      1807
Mild         408
Moderate     188
Severe        59
Name: count, dtype: int64

In [16]:
Target_remapper = {'No label': 'No label', 
                   'Normal': 'Normal',
                   'Mild': 'Not normal',
                   'Moderate': 'Not normal',
                   'Severe': 'Not normal',
                   'Present': 'Not normal'
                   }
if reduce_labels:
    labeled_documents['label'] = labeled_documents.label.map(Target_remapper)
    print(labeled_documents.label.value_counts(dropna=False))

label
No label      2538
Normal        1807
Not normal     655
Name: count, dtype: int64


In [17]:
labeled_documents

,text,label,_input_hash
0,Goede functie van R.V. en L.V. Nog bekijken me...,Normal,-1492643194
1,Goede L.V. functie en R.V. functie. Er is een ...,Normal,-1268308324
2,"Normale dimensie LV. prominent basale septum, ...",No label,-1752879130
3,Globaal redelijke systolische restfunctie. Voo...,No label,-1692751703
4,Goede systolische LV-functie. Matige cLVH. Gee...,No label,570163096
...,...,...,...
4995,De beperkte echovensters suggereren goede LV e...,Normal,1186042732
4996,TEE tape 191\r\nOnder 1 % lidocaine vlotte int...,No label,-478911984
4997,Technisch moeizaam echo.\r\n- De L.V. is niet ...,Normal,963390338
4998,Matig echo window.\r\nNiet gedilateerde LV met...,No label,-1897706486


In [18]:
# Expand with label columns
Target_maps = {Label:i for i,Label in enumerate(labeled_documents['label'].unique())}

In [19]:
Target_maps

{'Normal': 0, 'No label': 1, 'Not normal': 2}

In [20]:
# Load the train/test hashes
test_hashes = pd.read_csv('./test_echoid.csv', sep=',')
train_hashes = pd.read_csv('./train_echoid.csv', sep=',')
print(train_hashes.columns)

print(f"Train hashes: {train_hashes.input_hash.nunique()}")
print(f"Test hashes: {test_hashes.input_hash.nunique()}")

Index(['ECHO_StudyID', 'ECHO_StudyID.1', 'input_hash', 'task_hash'], dtype='object')
Train hashes: 96026
Test hashes: 24051


In [21]:
print(Target_maps.keys())

dict_keys(['Normal', 'No label', 'Not normal'])


In [22]:
# We now make DataSets (a special HuggingFace structure)
# assuming cross-validation

DF = labeled_documents
DF.columns = ['sentence', 'labels', '_input_hash']

label2id = Target_maps
id2label = {v:k for k,v in label2id.items()}
num_labels = len(label2id)
DF['labels'] = DF['labels'].map(label2id)


if filter_reports:
    DF = DF.assign(sentence = echo_utils.report_filter(DF.sentence, 
                                            flag_terms=FLAG_TERMS, 
                                            save_terms=SAVE_TERMS)[0])
    DF = DF.loc[DF.sentence.notna()]

if deabbreviate:
    DeAbber = deabber.deabber(model_type='sbert', 
                              abbreviations=ABBREVIATIONS['nl']['echocardiogram'], 
                              min_sim=0.5, top_k=10)
    DF = DF.assign(sentence=DeAbber.deabb(DF.sentence.values, TokenRadius=3))


In [23]:
if use_multilabel:
    _multilabels = {label2id[k]: [label2id[l] for l in v]
                    for k,v in MULTILABELS.items()}
else:
    _multilabels = None

In [24]:

# TODO: make proper
DFtrain = DF.loc[DF._input_hash.isin(train_hashes.input_hash), ['sentence', 'labels']]
DFtest = DF.loc[DF._input_hash.isin(test_hashes.input_hash), ['sentence', 'labels']]

print("Train labels:")
print(DFtrain.labels.value_counts())
print("Test labels:")
print(DFtest.labels.value_counts())

TrainSet = Dataset.from_pandas(DFtrain)
TestSet = Dataset.from_pandas(DFtest)

HF_DataSet = DatasetDict(
    {'train' : TrainSet,
     'test': TestSet,
    }
)

Tokenized_DataSet = HF_DataSet.map(lambda batch: tokenizer(batch, truncation=True, 
                                                                  padding=True, 
                                                                  max_length=256),
                                  input_columns='sentence',
                                  batched=True,
                                  remove_columns=['sentence'])

Train labels:
labels
1    1958
0    1431
2     520
Name: count, dtype: int64
Test labels:
labels
1    497
0    342
2    119
Name: count, dtype: int64


Map:   0%|          | 0/3909 [00:00<?, ? examples/s]

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

In [25]:
Tokenized_DataSet = (Tokenized_DataSet
                      #.map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
                      .map(lambda x: {"labels": 
                          multi_hot_encoding(x['labels'], 
                                             multilabels=_multilabels, 
                                             num_classes=num_labels)}, 
                           batched=True, remove_columns=['labels']))                      
                      #.rename_column("float_labels", "labels"))

Tokenized_DataSet.set_format("torch", 
                             columns=['input_ids', 'attention_mask', 'labels'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3909 [00:00<?, ? examples/s]

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

In [29]:
# https://colab.research.google.com/drive/1aue7x525rKy6yYLqqt-5Ll96qjQvpqS7#scrollTo=1eVCRpcLUW-y
# https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
medroberta_clf = AutoModelForSequenceClassification.from_pretrained("CLTL/MedRoBERTa.nl", 
                                                num_labels=len(id2label.keys()),
                                                problem_type='multi_label_classification',
                                                id2label=id2label,
                                                label2id=label2id)


c:\Users\bes3\AppData\Local\pypoetry\Cache\virtualenvs\echolabeler-DyKQcQCO-py3.10\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at CLTL/MedRoBERTa.nl and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from torchinfo import summary
summary(medroberta_clf)

Layer (type:depth-idx)                                       Param #
RobertaForSequenceClassification                             --
├─RobertaModel: 1-1                                          --
│    └─RobertaEmbeddings: 2-1                                --
│    │    └─Embedding: 3-1                                   39,936,000
│    │    └─Embedding: 3-2                                   394,752
│    │    └─Embedding: 3-3                                   768
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─RobertaEncoder: 2-2                                   --
│    │    └─ModuleList: 3-6                                  85,054,464
├─RobertaClassificationHead: 1-2                             --
│    └─Linear: 2-3                                           590,592
│    └─Dropout: 2-4                                          --
│    └─Linear: 2-5                                           2,307
To

## MedRoBERTa.nl -- Training setup

In [27]:
train_dir = "T:\\laupodteam/AIOS/Bram/data/tmp"
metric_name = 'f1_macro'
training_args = TrainingArguments(output_dir=train_dir,
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=num_epochs,
                                  learning_rate=5e-5,
                                  per_device_train_batch_size=16,
                                  weight_decay=0.01,
                                  per_device_eval_batch_size=10,
                                  load_best_model_at_end=True, 
                                  metric_for_best_model=metric_name)

In [31]:
trainer = Trainer(medroberta_clf,
                  training_args,
                  train_dataset = Tokenized_DataSet["train"],
                  eval_dataset = Tokenized_DataSet["test"],
                  tokenizer=tokenizer,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics)

## MedRoBERTa.nl -- run Training

In [408]:
trainer.train()

  0%|          | 0/1225 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.10540775954723358, 'eval_f1_macro': 0.9134137229403607, 'eval_f1_weighted': 0.9526689509950355, 'eval_prec_macro': 0.9406026561568023, 'eval_prec_weighted': 0.9536079685744389, 'eval_recall_macro': 0.8907472503835029, 'eval_recall_weighted': 0.9530271398747391, 'eval_roc_auc_macro': 0.9840262113286249, 'eval_roc_auc_weighted': 0.9858562572623008, 'eval_accuracy': 0.9498956158663883, 'eval_runtime': 4.8372, 'eval_samples_per_second': 198.049, 'eval_steps_per_second': 19.846, 'epoch': 1.0}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.09395516663789749, 'eval_f1_macro': 0.914843517480219, 'eval_f1_weighted': 0.9542215876731077, 'eval_prec_macro': 0.9117272157605955, 'eval_prec_weighted': 0.9594256835350793, 'eval_recall_macro': 0.9206477943632154, 'eval_recall_weighted': 0.9498956158663883, 'eval_roc_auc_macro': 0.9909293049109262, 'eval_roc_auc_weighted': 0.9899882726030039, 'eval_accuracy': 0.9498956158663883, 'eval_runtime': 4.9374, 'eval_samples_per_second': 194.029, 'eval_steps_per_second': 19.443, 'epoch': 2.0}
{'loss': 0.1413, 'grad_norm': 0.262879341840744, 'learning_rate': 2.959183673469388e-05, 'epoch': 2.04}


...

  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.08312099426984787, 'eval_f1_macro': 0.9498360507329767, 'eval_f1_weighted': 0.9727502877264206, 'eval_prec_macro': 0.9549569829921004, 'eval_prec_weighted': 0.9726893998375026, 'eval_recall_macro': 0.9448446459603422, 'eval_recall_weighted': 0.9728601252609603, 'eval_roc_auc_macro': 0.994079687176257, 'eval_roc_auc_weighted': 0.9936515243707174, 'eval_accuracy': 0.9728601252609603, 'eval_runtime': 4.8876, 'eval_samples_per_second': 196.005, 'eval_steps_per_second': 19.641, 'epoch': 4.0}
{'loss': 0.0223, 'grad_norm': 8.819168090820312, 'learning_rate': 9.183673469387756e-06, 'epoch': 4.08}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.08433438837528229, 'eval_f1_macro': 0.9516461020337991, 'eval_f1_weighted': 0.973193956443554, 'eval_prec_macro': 0.9605577175051468, 'eval_prec_weighted': 0.9736628085266955, 'eval_recall_macro': 0.9430809290109305, 'eval_recall_weighted': 0.9728601252609603, 'eval_roc_auc_macro': 0.994129173466613, 'eval_roc_auc_weighted': 0.9935770726766903, 'eval_accuracy': 0.9718162839248434, 'eval_runtime': 4.9123, 'eval_samples_per_second': 195.021, 'eval_steps_per_second': 19.543, 'epoch': 5.0}
{'train_runtime': 494.4517, 'train_samples_per_second': 39.529, 'train_steps_per_second': 2.477, 'train_loss': 0.06798359335685263, 'epoch': 5.0}


TrainOutput(global_step=1225, training_loss=0.06798359335685263, metrics={'train_runtime': 494.4517, 'train_samples_per_second': 39.529, 'train_steps_per_second': 2.477, 'total_flos': 2571275874746880.0, 'train_loss': 0.06798359335685263, 'epoch': 5.0})

In [409]:
trainer.eval_dataset = Tokenized_DataSet["test"]
trainer.evaluate()

  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.08433438837528229,
 'eval_f1_macro': 0.9516461020337991,
 'eval_f1_weighted': 0.973193956443554,
 'eval_prec_macro': 0.9605577175051468,
 'eval_prec_weighted': 0.9736628085266955,
 'eval_recall_macro': 0.9430809290109305,
 'eval_recall_weighted': 0.9728601252609603,
 'eval_roc_auc_macro': 0.994129173466613,
 'eval_roc_auc_weighted': 0.9935770726766903,
 'eval_accuracy': 0.9718162839248434,
 'eval_runtime': 5.0331,
 'eval_samples_per_second': 190.338,
 'eval_steps_per_second': 19.074,
 'epoch': 5.0}

In [410]:
#medroberta_clf_pipe = pipeline('text-classification', model=medroberta_clf, tokenizer=tokenizer)

In [411]:
# get current directory of .py file, i.e. NOT os.getcwd()
#os.path.dirname(os.path.realpath(__file__))